In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
from bs4 import BeautifulSoup
import requests
from time import sleep
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

/opt/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Task
#looking for mentions of the words above, and generates a csv that lists the keyword and the URL at which it was found. 
#This project is a bit more complicated than what we've done in the tutorials, 
#but only a little bit. It will require you to extract the links (found in "a" tags) and sort through them to make sure you are only getting the ones in the gallery. 
#You may need to look through the BeautifulSoup documentation to figure out how to get at the hrefs from those A tags. Note that Craigslist seems to be blocking Google IPs, 
#so you'll need to do this one from your own machine. Once you have a list of URLs, you'll need to crawl them one-by-one, and search the text of the ads for each of your keywords (see above).
#Note that while Craigslist has an API, you need to be scraping directly here to show your scraping skills. Remember to build in a sizeable "sleep" between each request. 

Step 1: Get the links for Garage Sales and Moving Sales

In [17]:
# Doing it with Selenium
driver = webdriver.Chrome(executable_path = '/Desktop/chromedriver')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [18]:
# Get the page with filters in key_words required and filter results daily
driver.get('https://phoenix.craigslist.org/search/gms?postedToday=1&query=*mattress*%20%7C%20*cabinet*%20%7C%20*wrench*#search=1~gallery~0~0')

In [22]:
print(driver.page_source)

<html class="cl-not-small-device cl-landscape"><head>
    
	<meta charset="UTF-8">
	<meta http-equiv="X-UA-Compatible" content="IE=Edge">
	<meta name="viewport" content="width=device-width,initial-scale=1">
	<meta property="og:site_name" content="craigslist">
	<meta name="twitter:card" content="preview">
	<meta property="og:title" content="phoenix garage &amp; moving sales &quot;*mattress* | *cabinet* | *wrench*&quot; -...">
	<meta name="description" content="phoenix garage &amp; moving sales &quot;*mattress* | *cabinet* | *wrench*&quot; - craigslist">
	<meta property="og:description" content="phoenix garage &amp; moving sales &quot;*mattress* | *cabinet* | *wrench*&quot; - craigslist">
	<meta property="og:url" content="https://phoenix.craigslist.org/search/gms?query=*mattress*+%7C+*cabinet*+%7C+*wrench*">
	<title>phoenix garage &amp; moving sales "*mattress* | *cabinet* | *wrench*"  - craigslist</title>
	<link rel="canonical" href="https://phoenix.craigslist.org/search/gms?query=*matt

In [6]:
import selenium
selenium.__version__

'4.8.0'

In [23]:
# Get all urls lists in the page source
all_links = driver.find_elements("tag name","a")
elems = driver.find_elements("xpath","//*[@href]")

In [24]:
# Put all urls in a list
urls =[]
for elem in elems:
    urls.append(elem.get_attribute("href"))
    sleep(5)

In [25]:
urls

['https://phoenix.craigslist.org/search/gms?query=*mattress*+%7C+*cabinet*+%7C+*wrench*',
 'https://www.craigslist.org/static/www/www-2023-02-16-17-05-3f0f531f13c8d37c17be5506a8eb248069e00430.css',
 'https://phoenix.craigslist.org/',
 'https://www.craigslist.org/about/FHA',
 'https://www.craigslist.org/about/safety',
 'https://www.craigslist.org/about/prohibited',
 'https://www.craigslist.org/about/recalled_items',
 'https://www.craigslist.org/about/scams',
 'https://phoenix.craigslist.org/nph/gms/d/phoenix-durable-medical-equipment/7590192090.html',
 'https://phoenix.craigslist.org/nph/gms/d/phoenix-durable-medical-equipment/7590192090.html',
 'https://phoenix.craigslist.org/nph/gms/d/phoenix-friday-sat-large-estate-sale/7590071646.html',
 'https://phoenix.craigslist.org/nph/gms/d/phoenix-friday-sat-large-estate-sale/7590071646.html',
 'https://lasvegas.craigslist.org/gms/d/henderson-estate-sale/7590094459.html',
 'https://lasvegas.craigslist.org/gms/d/henderson-estate-sale/7590094459

In [26]:
# Filter url links and only contain those opened as a post
urls_updated =[]
for i in range(len(urls)):
    if urls[i] not in urls_updated:
        if 'html' in urls[i] and 'phoenix.craigslist.org' in urls[i]:
            urls_updated.append(urls[i])


In [62]:
urls_updated

['https://phoenix.craigslist.org/nph/gms/d/phoenix-durable-medical-equipment/7590192090.html',
 'https://phoenix.craigslist.org/nph/gms/d/phoenix-friday-sat-large-estate-sale/7590071646.html']

Step 2: Check the urls one by one and find which keyword in the post

In [46]:
# put the result in a dictionary
my_dict = {}
for i in range(len(urls_updated)):
    thispage = requests.get(urls_updated[i])
    bs = BeautifulSoup(thispage.text,'html.parser')
    content = bs.get_text().lower()
    if "mattress" in content:
        my_dict["mattress"] = urls_updated[i]
    elif "cabinet" in content:
        my_dict["cabinet"] = urls_updated[i]
    else:
        my_dict["wrench"] = urls_updated[i]
    sleep(5)

In [47]:
my_dict

{'mattress': 'https://phoenix.craigslist.org/nph/gms/d/phoenix-durable-medical-equipment/7590192090.html',
 'cabinet': 'https://phoenix.craigslist.org/nph/gms/d/phoenix-friday-sat-large-estate-sale/7590071646.html'}

In [64]:
# quit the drive
driver.quit()

Step 3: Convert the dictionary to a pandas dataframe.

In [56]:
import pandas as pd 
df = pd.DataFrame.from_dict(my_dict,orient='index').reset_index()

In [60]:
df = df.rename(columns= {"index":"key_word",0: "url"})

In [61]:
# output
df.to_csv("craigslist.csv")